In [1]:
import os
import sys
sys.path.append('/mnt/c/Users/44yos/RacePrediction/resnet_win5')
sys.path.append('/usr/local/bin/chromedriver')
from data_setup.race_data.this_week_race_url import this_week_race_url
from data_setup.race_data.this_week_race_html import this_week_race_html
from data_setup.race_data.make_csv_from_this_week_html import make_csv_from_this_week_html
# from data_setup.race_data.make_csv_this_week_with_odds import make_csv_this_week_with_odds
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from os import path

from pickle import load

from models import resnet
from tensorflow.python import keras
from tensorflow.python.keras.models import load_model
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [2]:
this_week_race_url()

In [3]:
this_week_race_html()

In [4]:
os.makedirs('../data/csv/2022/Dec_2', exist_ok = True)
make_csv_from_this_week_html()

In [5]:
#make_csv_this_week_with_odds()

In [6]:
race_df = pd.read_csv("../data/csv/2022/Dec_2/12_11.csv", sep = ",")

In [7]:
print(len(race_df['race_id'].unique()))
print(race_df.shape)
print(race_df.head(200))

12
(164, 22)
             race_id race_round  race_title    race_distance weather  \
0    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
1    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
2    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
3    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
4    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
5    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
6    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
7    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
8    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
9    202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
10   202206050408&rf   \n\n8R\n   3歳以上1勝クラス     ダ2500m (右)\n       晴   
11   202206050409&rf   \n\n9R\n       チバテレ杯     ダ1800m (右)\n       晴   
12   202206050409&rf   \n\n9R\n       チバテレ杯     ダ18

In [8]:
odds = pd.DataFrame(columns = ["odds"])
odds["odds"] = [4.4,22.6,199.1,29.9,399.3,18.0,2.8,109.9,165.0,4.5,251.1,95.8,70.7,10.1,169.7,6.0,74.0,1.4,6.5,87.8,12.4,40.8,50.6,20.4,12.4,9.4,3.2,4.3,22.5,56.1,24.3,31.1,15.3,12.5,8.2,6.2,6.3,93.0,49.1,11.2,8.7,6.9,2.4,22.4,65.4,58.1,8.2,12.4,25.0,39.9,182.5,52.9,10.3,31.1,37.4,2.0,251.6,101.8,13.4,3.7,5.4,20.7,28.3,13.4,20.5,8.0,2.2,67.7,13.8,117.9,104.2,10.4,23.7,13.4,4.9,13.3,105.3,85.7,104.2,74.1,10.7,13.6,3.1,94.1,5.1,13.1,150.5,72.1,50.1,6.0,38.0,104.4,45.3,32.2,5.4,201.2,4.3,75.1,10.3,144.6,8.2,5.9,9.6,21.0,9.2,82.8,8.5,20.2,9.3,21.2,104.7,85.8,148.6,2.6,6.1,11.2,7.6,3.7,8.6,13.8,36.8,130.6,29.2,5.9,21.0,6.3,5.8,4.7,14.2,35.2,66.7,16.9,6.8,6.9,28.0,3.3,6.6,28.7,7.5,90.3,28.9,44.5,20.2,3.0,42.8,7.8,10.6,23.3,46.2,48.7,6.3,45.4,214.9,163.0,3.2,140.4,129.7,31.6,41.7,17.7,54.9,15.0,6.4,6.9]

In [9]:
print(odds.shape)
# print(odds)

(164, 1)


In [10]:
race_df["odds"] = odds["odds"]

In [11]:
race_df.tail(50)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
114,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,3,3,2017104932,牡5,57.0,1144,?,?,?,6.1,?
115,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,4,4,2017106570,セ5,57.0,1174,?,?,?,11.2,?
116,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,5,5,2018104797,牡4,57.0,1018,?,?,?,7.6,?
117,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,6,6,2017105339,牡5,57.0,1126,?,?,?,3.7,?
118,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,7,7,2015102127,セ7,57.0,1163,?,?,?,8.6,?
119,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,7,8,2018100410,牡4,57.0,1077,?,?,?,13.8,?
120,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,8,9,2018105326,牡4,57.0,1157,?,?,?,36.8,?
121,202209060408&rf,\n\n8R\n,3歳以上2勝クラス,芝2400m (右 外)\n,晴,良,13:55発走,2022-12-11,阪神,10,?,8,10,2018101321,牡4,57.0,5390,?,?,?,130.6,?
122,202209060409&rf,\n\n9R\n,オリオンS,芝2200m (右)\n,晴,良,14:25発走,2022-12-11,阪神,12,?,1,1,2018104395,牡4,57.0,1140,?,?,?,29.2,?
123,202209060409&rf,\n\n9R\n,オリオンS,芝2200m (右)\n,晴,良,14:25発走,2022-12-11,阪神,12,?,2,2,2017104141,牝5,55.0,1126,?,?,?,5.9,?


race_round

In [12]:
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [13]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
0,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?
1,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?
2,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?
3,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?
4,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?


race_rank

In [14]:
race_df["race_title"] = race_df["race_title"].replace('阪神JF','阪神JFG1')
# race_df["race_title"] = race_df["race_title"].replace('キーンランドC','キーンランドCG3')
# race_df["race_title"] = race_df["race_title"].replace('新潟2歳S','新潟2歳SG3')
# race_df["race_title"] = race_df["race_title"].replace('府中牝馬S','府中牝馬SG2')
# race_df["race_title"] = race_df["race_title"].replace('神戸新聞杯','神戸新聞杯G2')

In [15]:
race_rank = race_df["race_title"].str.extract('(G1|G2|G3)', expand = True)
race_rank.columns = {"race_rank"}

race_df = pd.concat([race_df, race_rank], axis = 1)
race_df.fillna(value = {'race_rank': 0}, inplace = True) # 元もデータに反映
race_df['race_rank'] = race_df['race_rank'].replace('G1', 3)
race_df['race_rank'] = race_df['race_rank'].replace('G2', 2)
race_df['race_rank'] = race_df['race_rank'].replace('G3', 1)

In [16]:
print("race_rank:", race_df["race_rank"].value_counts())

race_rank: 0    146
3     18
Name: race_rank, dtype: int64


In [17]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank
0,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0
1,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0
2,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0
3,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0
4,202206050408&rf,8,3歳以上1勝クラス,ダ2500m (右)\n,晴,良,13:40発走,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0


race_distance

In [18]:
race_df["race_distance"].unique()

array([' ダ2500m (右)\n', ' ダ1800m (右)\n', ' 芝1800m (右)\n', ' ダ1200m (右)\n',
       ' ダ1400m (左)\n', ' ダ1800m (左)\n', ' 芝2000m (左)\n', ' 芝1200m (左)\n',
       ' 芝2400m (右 外)\n', ' 芝2200m (右)\n', ' ダ2000m (右)\n',
       ' 芝1600m (右 外)\n'], dtype=object)

In [19]:
ground_type = race_df["race_distance"].str.extract('(芝|ダ)', expand = True)
circle = race_df["race_distance"].str.extract('(右|左)', expand = True)
distance = race_df["race_distance"].str.extract('(\d+)m', expand = True)

ground_type.columns = {"ground_type"}
circle.columns = {"circle"}
distance.columns = {"distance"}

race_df = pd.concat([race_df, ground_type], axis = 1)
race_df = pd.concat([race_df, circle], axis = 1)
race_df = pd.concat([race_df, distance], axis = 1)

race_df = pd.get_dummies(race_df, columns = ["ground_type", "circle"])

In [20]:
race_df["distance"] = race_df["distance"].astype(int)
race_df.drop(['race_distance'], axis = 1, inplace = True)

In [21]:
# # 存在しない右左回り適宜追加 ######### circle_左 or circle_右
add_circle = pd.DataFrame(columns = ["circle_右","circle_左"])
race_df = pd.concat([race_df,add_circle], axis = 1).fillna(0)

In [22]:
race_df.head(5)

,race_id,race_round,race_title,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左
0,202206050408&rf,8,3歳以上1勝クラス,晴,良,13:40発走,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0
1,202206050408&rf,8,3歳以上1勝クラス,晴,良,13:40発走,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0
2,202206050408&rf,8,3歳以上1勝クラス,晴,良,13:40発走,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0
3,202206050408&rf,8,3歳以上1勝クラス,晴,良,13:40発走,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0
4,202206050408&rf,8,3歳以上1勝クラス,晴,良,13:40発走,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0


weather

In [23]:
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '小倉', '晴')
race_df["weather"] = race_df["weather"].mask(race_df["place"] == '阪神', '曇')
race_df["weather"] = race_df["weather"].mask(race_df["place"] == '中山', '曇')
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '札幌', '晴')

In [24]:
weather = race_df["weather"].str.extract('(晴|曇|小雨|雨|小雪|雪)', expand = True)
weather.columns = {"weather_circumstance"}
race_df = pd.concat([race_df, weather], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["weather_circumstance"])
race_df.drop(['weather'], axis = 1, inplace = True)

In [25]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇
0,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1
1,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1
2,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1
3,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1
4,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1


In [26]:
# 存在しない天気適宜追加 #########
add_weather = pd.DataFrame(columns = ["weather_circumstance_小雨","weather_circumstance_雨","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_小雪","weather_circumstance_雪"])
race_df = pd.concat([race_df,add_weather], axis = 1).fillna(0)

In [27]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
1,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
2,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,202206050408&rf,8,3歳以上1勝クラス,良,13:40発走,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0


ground_condition

In [28]:
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '小倉', '良')
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '中山', '稍重')
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '中京', '稍重')
# race_df["ground_condition"] = "良"

In [29]:
race_df["ground_condition"] = race_df["ground_condition"].replace('(不良)', 4, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(稍重)', 2, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(重)', 3, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(良)', 1, regex = True)

In [30]:
print("ground_condition:", race_df["ground_condition"].value_counts())

ground_condition: 1    164
Name: ground_condition, dtype: int64


In [31]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202206050408&rf,8,3歳以上1勝クラス,1,13:40発走,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
1,202206050408&rf,8,3歳以上1勝クラス,1,13:40発走,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
2,202206050408&rf,8,3歳以上1勝クラス,1,13:40発走,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3,202206050408&rf,8,3歳以上1勝クラス,1,13:40発走,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,202206050408&rf,8,3歳以上1勝クラス,1,13:40発走,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0


datetime

In [32]:
race_df["date"] =  pd.to_datetime(race_df['date'])
race_df.drop(["time"], axis = 1, inplace = True)

In [33]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,中山,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,中山,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,中山,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
3,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,中山,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,中山,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0


place

In [34]:
race_df = pd.get_dummies(race_df, columns = ["place"])

In [35]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中京,place_中山,place_阪神
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0


In [36]:
# 存在しない場所追加 '札幌', '中京', '新潟', '函館', '小倉', '京都','東京','福島', '阪神' '中山'
add_place = pd.DataFrame(columns =['札幌','新潟','函館','小倉','京都','東京','福島'])
race_df = pd.concat([race_df,add_place], axis = 1).fillna(0)

In [37]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中京,place_中山,place_阪神,札幌,新潟,函館,小倉,京都,東京,福島
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,1,1,2018103437,牡4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,2,2,2018102807,牡4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,3,3,2019105979,牡3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,4,4,2018100121,牡4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,5,5,2019100577,牡3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


sex_age

In [38]:
sex = race_df["sex_and_age"].str.extract('(牡|牝|セ)', expand = True)
sex.columns = {"sex"}
race_df = pd.concat([race_df, sex], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["sex"])

race_df["sex_and_age"] = race_df["sex_and_age"].str.strip("牡牝セ\n")
race_df["sex_and_age"] = race_df["sex_and_age"].astype(int)
race_df = race_df.rename(columns = {"sex_and_age": "age"})

In [39]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中京,place_中山,place_阪神,札幌,新潟,函館,小倉,京都,東京,福島,sex_セ,sex_牝,sex_牡
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,1,1,2018103437,4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,2,2,2018102807,4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,3,3,2019105979,3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,4,4,2018100121,4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,5,5,2019100577,3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


burden_weight

In [44]:
race_df = race_df[race_df['burden_weight'] != '未定']

In [45]:
race_df["burden_weight"] = race_df["burden_weight"].astype(float)

odds

In [46]:
print(race_df["odds"].dtypes)

float64


In [47]:
race_df.to_csv("../data/csv/2022/Dec_2/refined_12_11_race.csv", index = False)

In [48]:
#race_df = pd.read_csv("../data/csv/2021/dec_4/refined_12_26_race.csv", sep = ",")

make_train_data

In [49]:
horse_df = pd.read_csv("../data/csv/horse_data/refined_horse_data.csv", sep = ",")
horse_f_df = pd.read_csv("../data/csv/horse_data/horse_featured_data.csv", sep = ",")

/home/yoshi/miniconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
race_df['order'] = race_df['order'].astype(str)
race_df['horse_id'] = race_df['horse_id'].astype(str)
race_df['jockey_id'] = race_df['jockey_id'].astype(str)
race_df['date'] = pd.to_datetime(race_df['date'])
print(race_df.shape)
print(race_df.dtypes)
race_df.head(3)

(162, 47)
race_id                            object
race_round                         object
race_title                         object
ground_condition                    int64
date                       datetime64[ns]
total_horse_number                  int64
order                              object
frame_number                        int64
horse_number                        int64
horse_id                           object
age                                 int64
burden_weight                     float64
jockey_id                          object
goal_time                          object
half_order                         object
last_time                          object
odds                              float64
horse_weight                       object
race_rank                           int64
distance                            int64
ground_type_ダ                       uint8
ground_type_芝                       uint8
circle_右                            uint8
circle_左                

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中京,place_中山,place_阪神,札幌,新潟,函館,小倉,京都,東京,福島,sex_セ,sex_牝,sex_牡
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,1,1,2018103437,4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,2,2,2018102807,4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,3,3,2019105979,3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [51]:
horse_df['order'] = horse_df['order'].astype(str)
horse_df['horse_id'] = horse_df['horse_id'].astype(str)
horse_df['date'] = pd.to_datetime(horse_df['date'])
print(horse_df.dtypes)
print(horse_df.shape)
horse_df.head(3)

horse_id                           object
father_id                          object
grandfather_id                     object
mother_id                          object
total_race_number                   int64
date                       datetime64[ns]
whole_horse_number                  int64
odds                              float64
order                              object
jockey_id                          object
burden_weight                     float64
race_distance                       int64
ground_condition                    int64
goal_time                         float64
half_order                        float64
last_time                         float64
horse_weight                      float64
weather_circumstance_小雨             int64
weather_circumstance_小雪             int64
weather_circumstance_晴              int64
weather_circumstance_曇              int64
weather_circumstance_雨              int64
weather_circumstance_雪              int64
main_place_その他                    

,horse_id,father_id,grandfather_id,mother_id,total_race_number,date,whole_horse_number,odds,order,jockey_id,burden_weight,race_distance,ground_condition,goal_time,half_order,last_time,horse_weight,weather_circumstance_小雨,weather_circumstance_小雪,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_雨,weather_circumstance_雪,main_place_その他,main_place_中京,main_place_中山,main_place_京都,main_place_函館,main_place_小倉,main_place_新潟,main_place_札幌,main_place_東京,main_place_福島,main_place_阪神,race_rank,ground_type_ダ,ground_type_芝,ground_type_障,horse_weight_dif
0,2019100026,000a011155,000a000178,2013106103,11,2022-09-11,13,8.4,7,1156,55.0,1400,2,84.8,0.423077,38.2,480.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,8.0
1,2019100026,000a011155,000a000178,2013106103,11,2022-08-27,16,27.9,4,1156,54.0,1700,1,106.1,0.250000,38.5,472.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-2.0
2,2019100026,000a011155,000a000178,2013106103,11,2022-07-17,15,7.1,7,732,54.0,1700,1,106.7,0.333333,38.7,474.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,-4.0


In [52]:
horse_f_df['horse_id'] = horse_f_df['horse_id'].astype(str)
print(horse_f_df.dtypes)
print(horse_f_df.shape)
horse_f_df.head(3)

horse_id           object
grass_win_rate    float64
dart_win_rate     float64
win_rate          float64
dtype: object
(61856, 4)


,horse_id,grass_win_rate,dart_win_rate,win_rate
0,2019100026,0.0,0.090909,0.090909
1,2019100053,0.0,0.400000,0.400000
2,2019100055,0.0,0.333333,0.166667


In [53]:
pedigree_map = pd.read_csv("../data/csv/horse_data/pedigree_map.csv", sep = ",")
pedigree_map['horse_id'] = pedigree_map['horse_id'].astype(str)
print(pedigree_map.dtypes)

horse_id               object
f_grass_win_rate      float64
f_dart_win_rate       float64
f_win_rate            float64
g_f_grass_win_rate    float64
g_f_dart_win_rate     float64
g_f_win_rate          float64
m_grass_win_rate      float64
m_dart_win_rate       float64
m_win_rate            float64
dtype: object


add_pedigree_map to race_df

In [54]:
race_df = pd.merge(race_df, pedigree_map, on = "horse_id", how = "left").fillna(0)

In [55]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中京,place_中山,place_阪神,札幌,新潟,函館,小倉,京都,東京,福島,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,g_f_dart_win_rate,g_f_win_rate,m_grass_win_rate,m_dart_win_rate,m_win_rate
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,1,1,2018103437,4,57.0,1093,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.5,0.769231,0.733333,0.142857,0.8,0.416667,0.000000,0.000000,0.000000
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,2,2,2018102807,4,57.0,726,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.086957,0.000000,0.083333
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,3,3,2019105979,3,54.0,1188,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.5,0.000000,0.500000,0.533333,0.0,0.533333,0.000000,0.230769,0.150000
3,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,4,4,2018100121,4,57.0,422,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,5,5,2019100577,3,55.0,1138,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.037037,0.033333


add previous race

In [56]:
#delete unnecessary columns in horse_df
horse_df.drop(["father_id", "grandfather_id", "mother_id", "total_race_number"], axis = 1, inplace = True)

In [57]:
tmp_date_race_data = race_df[["date", "horse_id", "jockey_id"]]

In [58]:
#function 
def make_one_horse_data(tmp_date_horse_data, one_horse_df):
    one_horse_df = one_horse_df.sort_values('date', ascending = False) #False = 近いやつが上
    one_horse_df["same_jockey"] = (tmp_date_horse_data["jockey_id"] == one_horse_df["jockey_id"]) * 1.0
   
    
    one_horse_df = one_horse_df[(one_horse_df["date"] < tmp_date_horse_data["date"])] #レース前を抽出
    
    
    if (len(one_horse_df) <= 2):
        n = len(one_horse_df) 
    else:
        n = 3
    
    #print("one_horse_df")
    #print(one_horse_df)
    
    result_df = one_horse_df.copy()
    result_df.columns = result_df.columns + "_1"
    #print("result_df")
    #print(result_df)
    for i in range(1,n):
        shift_df = one_horse_df.shift(-i)
        #print("shift_df")
        #print(shift_df)
        shift_df.drop(["date"], axis = 1, inplace = True)
        shift_df.drop(["jockey_id"], axis = 1, inplace = True)
        shift_df.columns = shift_df.columns + "_" + str(i+1)
        result_df = pd.concat([result_df, shift_df], axis = 1)
        #print("result_df")
        #print(result_df)
    
    
    return result_df.head(1)                  #result_df.dropna()
        

In [59]:
one_data = tmp_date_race_data.iloc[0].copy()
one_horse_data = horse_df[horse_df["horse_id"] == "2018106606"].copy()
#print(one_data)
#print(one_horse_data)
tmp_race_df = make_one_horse_data(one_data, one_horse_data)
#print(tmp_race_df)

In [60]:
# うまくいくけど時間かかるやつ

for index, tmp in race_df.iterrows():
    tmp_race_df = tmp_race_df.append(make_one_horse_data(tmp, horse_df[horse_df["horse_id"] == tmp["horse_id"]]))


In [61]:
print(tmp_race_df.head(10))

       horse_id_1     date_1  whole_horse_number_1  odds_1 order_1  \
32057  2018106606 2022-06-11                     8    54.5       7   
24947  2018103437 2022-10-16                    12    18.4       8   
23857  2018102807 2022-06-26                    16    10.6       9   
6150   2019105979 2022-10-08                    13     4.9       2   
456    2019100577 2022-10-01                    16   173.2      16   
3316   2019103728 2022-10-08                    13    11.9       3   
45419  2017105117 2022-09-19                    16     4.8       5   
23274  2018102398 2022-10-23                    10    16.3       5   
21454  2018101171 2022-07-02                    13    45.9       9   
2985   2019103346 2022-09-17                    15     9.2       4   

      jockey_id_1  burden_weight_1  race_distance_1  ground_condition_1  \
32057        1135             55.0             2000                   1   
24947        1182             56.0             1800                   1   
2385

In [62]:
# tmp_race_df.to_pickle("../data/csv/2021/tmp_race_12_26_df.pkl")
#tmp_race_df = pd.read_pickle("../data/csv/2021/tmp_race_12_25_df.pkl")

In [63]:
#delete horse1,2,3_id
tmp_race_df.drop(['horse_id_2','horse_id_3'], axis = 1, inplace = True)

In [64]:
#tmp_race_df = tmp_race_df.dropna()
tmp_race_df["whole_horse_number_2"] = tmp_race_df["whole_horse_number_2"].fillna(tmp_race_df["whole_horse_number_1"])
tmp_race_df["odds_2"] = tmp_race_df["odds_2"].fillna(tmp_race_df["odds_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["burden_weight_2"] = tmp_race_df["burden_weight_2"].fillna(tmp_race_df["burden_weight_1"])
tmp_race_df["race_distance_2"] = tmp_race_df["race_distance_2"].fillna(tmp_race_df["race_distance_1"])
tmp_race_df["ground_condition_2"] = tmp_race_df["ground_condition_2"].fillna(tmp_race_df["ground_condition_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["goal_time_2"] = tmp_race_df["goal_time_2"].fillna(tmp_race_df["goal_time_1"])
#tmp_race_df["velocity_2"] = tmp_race_df["velocity_2"].fillna(tmp_race_df["velocity_1"])
tmp_race_df["half_order_2"] = tmp_race_df["half_order_2"].fillna(tmp_race_df["half_order_1"])
tmp_race_df["last_time_2"] = tmp_race_df["last_time_2"].fillna(tmp_race_df["last_time_1"])
tmp_race_df["horse_weight_2"] = tmp_race_df["horse_weight_2"].fillna(tmp_race_df["horse_weight_1"])
tmp_race_df["weather_circumstance_小雨_2"] = tmp_race_df["weather_circumstance_小雨_2"].fillna(tmp_race_df["weather_circumstance_小雨_1"])
tmp_race_df["weather_circumstance_小雪_2"] = tmp_race_df["weather_circumstance_小雪_2"].fillna(tmp_race_df["weather_circumstance_小雪_1"])
tmp_race_df["weather_circumstance_晴_2"] = tmp_race_df["weather_circumstance_晴_2"].fillna(tmp_race_df["weather_circumstance_晴_1"])
tmp_race_df["weather_circumstance_曇_2"] = tmp_race_df["weather_circumstance_曇_2"].fillna(tmp_race_df["weather_circumstance_曇_1"])
tmp_race_df["weather_circumstance_雨_2"] = tmp_race_df["weather_circumstance_雨_2"].fillna(tmp_race_df["weather_circumstance_雨_1"])
tmp_race_df["weather_circumstance_雪_2"] = tmp_race_df["weather_circumstance_雪_2"].fillna(tmp_race_df["weather_circumstance_雪_1"])
tmp_race_df["main_place_その他_2"] = tmp_race_df["main_place_その他_2"].fillna(tmp_race_df["main_place_その他_1"])
tmp_race_df["main_place_中京_2"] = tmp_race_df["main_place_中京_2"].fillna(tmp_race_df["main_place_中京_1"])
tmp_race_df["main_place_中山_2"] = tmp_race_df["main_place_中山_2"].fillna(tmp_race_df["main_place_中山_1"])
tmp_race_df["main_place_京都_2"] = tmp_race_df["main_place_京都_2"].fillna(tmp_race_df["main_place_京都_1"])
tmp_race_df["main_place_函館_2"] = tmp_race_df["main_place_函館_2"].fillna(tmp_race_df["main_place_函館_1"])
tmp_race_df["main_place_小倉_2"] = tmp_race_df["main_place_小倉_2"].fillna(tmp_race_df["main_place_小倉_1"])
tmp_race_df["main_place_新潟_2"] = tmp_race_df["main_place_新潟_2"].fillna(tmp_race_df["main_place_新潟_1"])
tmp_race_df["main_place_札幌_2"] = tmp_race_df["main_place_札幌_2"].fillna(tmp_race_df["main_place_札幌_1"])
tmp_race_df["main_place_東京_2"] = tmp_race_df["main_place_東京_2"].fillna(tmp_race_df["main_place_東京_1"])
tmp_race_df["main_place_福島_2"] = tmp_race_df["main_place_福島_2"].fillna(tmp_race_df["main_place_福島_1"])
tmp_race_df["main_place_阪神_2"] = tmp_race_df["main_place_阪神_2"].fillna(tmp_race_df["main_place_阪神_1"])
tmp_race_df["race_rank_2"] = tmp_race_df["race_rank_2"].fillna(tmp_race_df["race_rank_1"])
tmp_race_df["ground_type_ダ_2"] = tmp_race_df["ground_type_ダ_2"].fillna(tmp_race_df["ground_type_ダ_1"])
tmp_race_df["ground_type_芝_2"] = tmp_race_df["ground_type_芝_2"].fillna(tmp_race_df["ground_type_芝_1"])
tmp_race_df["ground_type_障_2"] = tmp_race_df["ground_type_障_2"].fillna(tmp_race_df["ground_type_障_1"])
tmp_race_df["horse_weight_dif_2"] = tmp_race_df["horse_weight_dif_2"].fillna(tmp_race_df["horse_weight_dif_1"])
tmp_race_df["same_jockey_2"] = tmp_race_df["same_jockey_2"].fillna(tmp_race_df["same_jockey_1"])

tmp_race_df["whole_horse_number_3"] = tmp_race_df["whole_horse_number_3"].fillna(tmp_race_df["whole_horse_number_2"])
tmp_race_df["odds_3"] = tmp_race_df["odds_3"].fillna(tmp_race_df["odds_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["burden_weight_3"] = tmp_race_df["burden_weight_3"].fillna(tmp_race_df["burden_weight_2"])
tmp_race_df["race_distance_3"] = tmp_race_df["race_distance_3"].fillna(tmp_race_df["race_distance_2"])
tmp_race_df["ground_condition_3"] = tmp_race_df["ground_condition_3"].fillna(tmp_race_df["ground_condition_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["goal_time_3"] = tmp_race_df["goal_time_3"].fillna(tmp_race_df["goal_time_2"])
#tmp_race_df["velocity_3"] = tmp_race_df["velocity_3"].fillna(tmp_race_df["velocity_2"])
tmp_race_df["half_order_3"] = tmp_race_df["half_order_3"].fillna(tmp_race_df["half_order_2"])
tmp_race_df["last_time_3"] = tmp_race_df["last_time_3"].fillna(tmp_race_df["last_time_2"])
tmp_race_df["horse_weight_3"] = tmp_race_df["horse_weight_3"].fillna(tmp_race_df["horse_weight_2"])
tmp_race_df["weather_circumstance_小雨_3"] = tmp_race_df["weather_circumstance_小雨_3"].fillna(tmp_race_df["weather_circumstance_小雨_2"])
tmp_race_df["weather_circumstance_小雪_3"] = tmp_race_df["weather_circumstance_小雪_3"].fillna(tmp_race_df["weather_circumstance_小雪_2"])
tmp_race_df["weather_circumstance_晴_3"] = tmp_race_df["weather_circumstance_晴_3"].fillna(tmp_race_df["weather_circumstance_晴_2"])
tmp_race_df["weather_circumstance_曇_3"] = tmp_race_df["weather_circumstance_曇_3"].fillna(tmp_race_df["weather_circumstance_曇_2"])
tmp_race_df["weather_circumstance_雨_3"] = tmp_race_df["weather_circumstance_雨_3"].fillna(tmp_race_df["weather_circumstance_雨_2"])
tmp_race_df["weather_circumstance_雪_3"] = tmp_race_df["weather_circumstance_雪_3"].fillna(tmp_race_df["weather_circumstance_雪_2"])
tmp_race_df["main_place_その他_3"] = tmp_race_df["main_place_その他_3"].fillna(tmp_race_df["main_place_その他_2"])
tmp_race_df["main_place_中京_3"] = tmp_race_df["main_place_中京_3"].fillna(tmp_race_df["main_place_中京_2"])
tmp_race_df["main_place_中山_3"] = tmp_race_df["main_place_中山_3"].fillna(tmp_race_df["main_place_中山_2"])
tmp_race_df["main_place_京都_3"] = tmp_race_df["main_place_京都_3"].fillna(tmp_race_df["main_place_京都_2"])
tmp_race_df["main_place_函館_3"] = tmp_race_df["main_place_函館_3"].fillna(tmp_race_df["main_place_函館_2"])
tmp_race_df["main_place_小倉_3"] = tmp_race_df["main_place_小倉_3"].fillna(tmp_race_df["main_place_小倉_2"])
tmp_race_df["main_place_新潟_3"] = tmp_race_df["main_place_新潟_3"].fillna(tmp_race_df["main_place_新潟_2"])
tmp_race_df["main_place_札幌_3"] = tmp_race_df["main_place_札幌_3"].fillna(tmp_race_df["main_place_札幌_2"])
tmp_race_df["main_place_東京_3"] = tmp_race_df["main_place_東京_3"].fillna(tmp_race_df["main_place_東京_2"])
tmp_race_df["main_place_福島_3"] = tmp_race_df["main_place_福島_3"].fillna(tmp_race_df["main_place_福島_2"])
tmp_race_df["main_place_阪神_3"] = tmp_race_df["main_place_阪神_3"].fillna(tmp_race_df["main_place_阪神_2"])
tmp_race_df["race_rank_3"] = tmp_race_df["race_rank_3"].fillna(tmp_race_df["race_rank_2"])
tmp_race_df["ground_type_ダ_3"] = tmp_race_df["ground_type_ダ_3"].fillna(tmp_race_df["ground_type_ダ_2"])
tmp_race_df["ground_type_芝_3"] = tmp_race_df["ground_type_芝_3"].fillna(tmp_race_df["ground_type_芝_2"])
tmp_race_df["ground_type_障_3"] = tmp_race_df["ground_type_障_3"].fillna(tmp_race_df["ground_type_障_2"])
tmp_race_df["horse_weight_dif_3"] = tmp_race_df["horse_weight_dif_3"].fillna(tmp_race_df["horse_weight_dif_2"])
tmp_race_df["same_jockey_3"] = tmp_race_df["same_jockey_3"].fillna(tmp_race_df["same_jockey_2"])



In [65]:
print(tmp_race_df.shape)
tmp_race_df.isnull().sum()

(133, 102)


horse_id_1                   0
date_1                       0
whole_horse_number_1         0
odds_1                       0
order_1                      0
jockey_id_1                  0
burden_weight_1              0
race_distance_1              0
ground_condition_1           0
goal_time_1                  0
half_order_1                 0
last_time_1                  0
horse_weight_1               0
weather_circumstance_小雨_1    0
weather_circumstance_小雪_1    0
weather_circumstance_晴_1     0
weather_circumstance_曇_1     0
weather_circumstance_雨_1     0
weather_circumstance_雪_1     0
main_place_その他_1             0
main_place_中京_1              0
main_place_中山_1              0
main_place_京都_1              0
main_place_函館_1              0
main_place_小倉_1              0
main_place_新潟_1              0
main_place_札幌_1              0
main_place_東京_1              0
main_place_福島_1              0
main_place_阪神_1              0
race_rank_1                  0
ground_type_ダ_1              0
ground_t

In [66]:
race_df = pd.merge(race_df, tmp_race_df, left_on = ["horse_id"], right_on = ["horse_id_1"], how = "left")

same_jockey

In [67]:
same_jockey = pd.DataFrame(columns = ["same_jockey"])
same_jockey["same_jockey"] = (race_df["jockey_id_1"] == race_df["jockey_id"]) * 1.0 

race_df = pd.concat([race_df,same_jockey], axis = 1)
race_df.drop(["jockey_id"], axis = 1,inplace = True)

In [68]:
race_df.drop(['horse_id_1','date_1'], axis = 1, inplace = True)

In [69]:
race_df["horse_id"] = race_df["horse_id"].drop_duplicates()
race_df = race_df.dropna(subset = {"horse_id"})

In [70]:
print(race_df.shape)
race_df.head(5)

(162, 156)


,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中京,place_中山,place_阪神,札幌,新潟,函館,小倉,京都,東京,福島,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,...,main_place_中京_2,main_place_中山_2,main_place_京都_2,main_place_函館_2,main_place_小倉_2,main_place_新潟_2,main_place_札幌_2,main_place_東京_2,main_place_福島_2,main_place_阪神_2,race_rank_2,ground_type_ダ_2,ground_type_芝_2,ground_type_障_2,horse_weight_dif_2,same_jockey_2,whole_horse_number_3,odds_3,order_3,burden_weight_3,race_distance_3,ground_condition_3,goal_time_3,half_order_3,last_time_3,horse_weight_3,weather_circumstance_小雨_3,weather_circumstance_小雪_3,weather_circumstance_晴_3,weather_circumstance_曇_3,weather_circumstance_雨_3,weather_circumstance_雪_3,main_place_その他_3,main_place_中京_3,main_place_中山_3,main_place_京都_3,main_place_函館_3,main_place_小倉_3,main_place_新潟_3,main_place_札幌_3,main_place_東京_3,main_place_福島_3,main_place_阪神_3,race_rank_3,ground_type_ダ_3,ground_type_芝_3,ground_type_障_3,horse_weight_dif_3,same_jockey_3,same_jockey
0,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,1,1,2018103437,4,57.0,?,?,?,4.4,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.5,0.769231,0.733333,0.142857,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-4.0,0.0,15.0,31.8,7,54.0,1800.0,1.0,114.2,0.516667,40.4,504.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0
1,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,2,2,2018102807,4,57.0,?,?,?,22.6,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,11.0,8.2,9,57.0,2100.0,3.0,134.1,0.704545,36.5,488.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-4.0,0.0,1.0
2,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,3,3,2019105979,3,54.0,?,?,?,199.1,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.5,0.000000,0.500000,0.533333,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-6.0,1.0,14.0,15.1,11,52.0,2000.0,1.0,122.0,0.892857,36.3,486.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0
3,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,4,4,2018100121,4,57.0,?,?,?,29.9,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,202206050408&rf,8,3歳以上1勝クラス,1,2022-12-11,11,?,5,5,2019100577,3,55.0,?,?,?,399.3,?,0,2500,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,41.6,14,56.0,1800.0,1.0,117.1,0.650000,41.6,466.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-2.0,0.0,1.0


In [71]:
race_df.to_csv("../data/csv/2022/Dec_2/12_11_data.csv", index = False)